## Load libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
sys.path.append("../libs")
sys.path.append("../")
from definitions import ROOT_DIR
import utils as utils

# Set pandas options
# Not use scientific notation and use two decimal places and use comma as thousand separator
pd.options.display.float_format = '{:,.2f}'.format

# ETL

## 1. Load dataset

In [3]:
# Load dataset from data_files/internet.xlsx - sheet: 'Ingresos '
df = utils.get_xls_sheet_data('/data_files/internet.xlsx', 'Ingresos ')
df.head()

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2024,2,"442,032,166.69",Abr-Jun 2024
1,2024,1,"346,198,986.13",Ene-Mar 2024
2,2023,4,"167,376,014.80",Oct-Dic 2023
3,2023,3,"133,106,593.41",Jul-Sept 2023
4,2023,2,"118,060,280.28",Jun-Mar 2023


## 2. Handling missing values

### Look for missing values in all cells

In [4]:
#Find missing values
df.isnull().sum()

Año                          0
Trimestre                    0
Ingresos (miles de pesos)    0
Periodo                      0
dtype: int64

#### There's no missing values

## 3. Look for Duplicates

### Find duplicates for complete rows

In [5]:
#Find duplicates by complete row
df.duplicated().sum()

0

#### There's no complete duplicated rows

### Fin duplicated rows for year and quarter ('Trimestre')

In [6]:
#Find duplicated rows by row, for year and quarter
df.duplicated(subset=['Año', 'Trimestre']).sum()

0

#### There's no duplicated rows for year and quarter

## 4. Finding outliers

In [7]:
#Finding outliers
df.describe()

,Año,Trimestre,Ingresos (miles de pesos)
count,42.00,42.00,42.00
mean,"2,019.00",2.45,"50,016,480.78"
std,3.73,1.13,"87,102,080.46"
min,"2,014.00",1.00,"2,984,054.21"
25%,"2,016.00",1.25,"7,055,326.25"
50%,"2,019.00",2.00,"20,475,265.73"
75%,"2,021.00",3.00,"44,850,901.45"
max,"2,033.00",4.00,"442,032,166.69"


#### In the statistics, we can see that there are outliers values for the column year ("Año"), because the maximum value is 2033 and the data is historical. Now we will review the year column in detail.

In [8]:
df.groupby(['Año','Trimestre']).sum()

Ingresos (miles de pesos)        Periodo
Año  Trimestre                                          
2014 1                       2,984,054.21   Ene-Mar 2014
     2                       3,270,816.20   Abr-Jun 2014
     3                       3,478,637.74  Jul-Sept 2014
     4                       3,950,440.78   Oct-Dic 2014
2015 1                       4,876,385.32   Ene-Mar 2015
     2                       4,701,790.72   Abr-Jun 2015
     3                       5,153,738.88  Jul-Sept 2015
     4                       5,376,899.21   Oct-Dic 2015
2016 1                       5,936,844.89   Ene-Mar 2016
     2                       6,534,240.60   Abr-Jun 2016
     3                       6,912,441.54  Jul-Sept 2016
     4                       7,483,980.39   Oct-Dic 2016
2017 1                       8,701,201.42   Ene-Mar 2017
     2                       9,678,646.93   Abr-Jun 2017
     3                      10,065,997.86  Jul-Sept 2017
     4                      11,226,127.39   Oct-Dic 2017
2018 1                      13,171,459.23   Ene-Mar 2018
     2                      14,319,467.46   Abr-Jun 2018
     3                      15,410,980.69  Jul-Sept 2018
     4                      16,637,686.59   Oct-Dic 2018
2019 1                      20,207,413.77   Ene-Mar 2019
     2                      20,743,117.69   Abr-Jun 2019
     3                      21,652,748.44  Jul-Sept 2019
     4                      24,169,250.70   Oct-Dic 2019
2020 1                      29,946,216.45   Ene-Mar 2020
     2                      32,102,475.99   Abr-Jun 2020
     3                      31,997,444.53  Jul-Sept 2020
     4                      33,539,702.72   Oct-Dic 2020
2021 1                      36,676,371.36   Ene-Mar 2021
     2                      38,239,666.63   Abr-Jun 2021
     3                      42,999,944.29  Jul-Sept 2021
     4                      45,467,887.17   Oct-Dic 2021
2022 1                      55,219,313.01   Ene-Mar 2022
     2                      60,419,500.62   Abr-Jun 2022
     3                      64,086,784.33  Jul-Sept 2022
     4                      72,443,482.99   Oct-Dic 2022
2023 2                     118,060,280.28   Jun-Mar 2023
     3                     133,106,593.41  Jul-Sept 2023
     4                     167,376,014.80   Oct-Dic 2023
2024 1                     346,198,986.13   Ene-Mar 2024
     2                     442,032,166.69   Abr-Jun 2024
2033 1                     104,134,992.81   Ene-Mar 2023

#### We can see there's no 1st quarter data for 20**2**3 and we have 1st quarter data for 20**3**3. I think there is a typing error and the correct year should be 2023. And in the column "Periodo" we have "End-Mar 2023", this confirms the hypothesis.  We will correct this error.

In [9]:
#Change year 2033 to 2023
df['Año'] = df['Año'].replace(2033, 2023)
df.groupby(['Año','Trimestre']).sum()

Ingresos (miles de pesos)        Periodo
Año  Trimestre                                          
2014 1                       2,984,054.21   Ene-Mar 2014
     2                       3,270,816.20   Abr-Jun 2014
     3                       3,478,637.74  Jul-Sept 2014
     4                       3,950,440.78   Oct-Dic 2014
2015 1                       4,876,385.32   Ene-Mar 2015
     2                       4,701,790.72   Abr-Jun 2015
     3                       5,153,738.88  Jul-Sept 2015
     4                       5,376,899.21   Oct-Dic 2015
2016 1                       5,936,844.89   Ene-Mar 2016
     2                       6,534,240.60   Abr-Jun 2016
     3                       6,912,441.54  Jul-Sept 2016
     4                       7,483,980.39   Oct-Dic 2016
2017 1                       8,701,201.42   Ene-Mar 2017
     2                       9,678,646.93   Abr-Jun 2017
     3                      10,065,997.86  Jul-Sept 2017
     4                      11,226,127.39   Oct-Dic 2017
2018 1                      13,171,459.23   Ene-Mar 2018
     2                      14,319,467.46   Abr-Jun 2018
     3                      15,410,980.69  Jul-Sept 2018
     4                      16,637,686.59   Oct-Dic 2018
2019 1                      20,207,413.77   Ene-Mar 2019
     2                      20,743,117.69   Abr-Jun 2019
     3                      21,652,748.44  Jul-Sept 2019
     4                      24,169,250.70   Oct-Dic 2019
2020 1                      29,946,216.45   Ene-Mar 2020
     2                      32,102,475.99   Abr-Jun 2020
     3                      31,997,444.53  Jul-Sept 2020
     4                      33,539,702.72   Oct-Dic 2020
2021 1                      36,676,371.36   Ene-Mar 2021
     2                      38,239,666.63   Abr-Jun 2021
     3                      42,999,944.29  Jul-Sept 2021
     4                      45,467,887.17   Oct-Dic 2021
2022 1                      55,219,313.01   Ene-Mar 2022
     2                      60,419,500.62   Abr-Jun 2022
     3                      64,086,784.33  Jul-Sept 2022
     4                      72,443,482.99   Oct-Dic 2022
2023 1                     104,134,992.81   Ene-Mar 2023
     2                     118,060,280.28   Jun-Mar 2023
     3                     133,106,593.41  Jul-Sept 2023
     4                     167,376,014.80   Oct-Dic 2023
2024 1                     346,198,986.13   Ene-Mar 2024
     2                     442,032,166.69   Abr-Jun 2024

#### The behaviour of the income ("Ingresos (miles de pesos)"), have a consistent growth. Although the first quarter of 2024 almost doubles the previous quarter, it may be due to the volatility of the currency in Argentina, it will be explored in detail in the EDA section.

## 5. Data Types

### We will review the data types for each column.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Año                        42 non-null     int64  
 1   Trimestre                  42 non-null     int64  
 2   Ingresos (miles de pesos)  42 non-null     float64
 3   Periodo                    42 non-null     object 
dtypes: float64(1), int64(2), object(1)
memory usage: 1.4+ KB


#### The data types are consistent with the data provided. Just the column "Periodo" have an object data type, and have strings values. We should change the data type to string, but since the data is redundant with the quarter ("trimestre") column, we will drop it.

In [11]:
#drop Periodo column
df = df.drop('Periodo', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Año                        42 non-null     int64  
 1   Trimestre                  42 non-null     int64  
 2   Ingresos (miles de pesos)  42 non-null     float64
dtypes: float64(1), int64(2)
memory usage: 1.1 KB


## New columns

### Create a new column with the year and quarter

In [12]:
#Create a new column with the quarter and year
df['Periodo'] = (df['Año'].astype(str) + 'T' + df['Trimestre'].astype(str)).astype('string')
df.head()

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2024,2,"442,032,166.69",2024T2
1,2024,1,"346,198,986.13",2024T1
2,2023,4,"167,376,014.80",2023T4
3,2023,3,"133,106,593.41",2023T3
4,2023,2,"118,060,280.28",2023T2


# Save dataset

In [13]:
df.to_parquet(ROOT_DIR + '/data_files/ingresos_clean.parquet')